In [ ]:
# Import our libraries
import pandas as pd
import numpy as np
# Import sklearn libraries
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve, auc, make_scorer, confusion_matrix, f1_score, fbeta_score
# Import the Naive Bayes, logistic regression, Bagging, RandomForest, AdaBoost, GradientBoost, Decision Trees and SVM Classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#plt.style.use('bmh')
#plt.style.use('ggplot')
plt.style.use('seaborn-notebook')
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,PReLU

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.4MB/s 


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
%cd /content/drive/MyDrive/yolov3

/content/drive/MyDrive/yolov3


In [ ]:
train=pd.read_csv('traind.csv')
test=pd.read_csv('testd.csv')

In [ ]:
X = train.drop(["label"],axis = 1).values
Y= train["label"].values
X_test=test.values

In [ ]:
!pip install utils
import utils

In [ ]:
from keras.utils import to_categorical

In [ ]:
X=X/255.0
X_test=X_test/255.0

In [ ]:
Y1=to_categorical(Y,num_classes=10)

In [ ]:
X=X.reshape(len(X),28,28,1)
X_test=X_test.reshape(len(X_test),28,28,1)

In [ ]:
def build_model(hp):
    
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [ ]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=3,executions_per_trial=2,directory='ou1a',project_name="nistdta")

In [ ]:
Y1.reshape(42000,10)

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
tuner.search(X, Y1, epochs=6, validation_split=0.1)

Trial 3 Complete [00h 01m 14s]
val_accuracy: 0.9872618913650513

Best val_accuracy So Far: 0.9872618913650513
Total elapsed time: 00h 02m 57s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.fit(X,Y1)

1313/1313 [==============================] - 7s 5ms/step - loss: 0.0134 - accuracy: 0.9954


In [ ]:
Y_test=best_model.predict(X_test)

In [ ]:
test_pred = np.argmax(Y_test,axis=1)


In [ ]:
p=[]
for i in range(1,28001):
  p.append(i)
p=np.asarray(p)  

In [ ]:
submission = pd.DataFrame({
        "ImageId": p,
        "Label": test_pred})

In [ ]:
submission.to_csv('final.csv',index=False)